# Train with RAPIDS

## Prerequisites

- Create an Azure ML Workspace and setup environmnet on local computer following the steps in [Azure README.md](https://github.com/rapidsai/cloud-ml-examples/blob/master/azure/README.md)

In [ ]:
# verify installation and check Azure ML SDK version
import azureml.core

print('SDK version:', azureml.core.VERSION)

## Create a FileDataset

In this example, we will use 20 million rows (samples) of the [airline dataset](http://kt.ijs.si/elena_ikonomovska/data.html). The [FileDataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.filedataset?view=azure-ml-py) below references parquet files that have been uploaded to a public [Azure Blob storage](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-blobs-overview), you can download to your local computer or mount the files to your AML compute.

In [ ]:
from azureml.core.dataset import Dataset
airline_ds = Dataset.File.from_files('https://airlinedataset.blob.core.windows.net/airline-20m/*')

# larger dataset (10 years of airline data) is also available for multi-GPU option
# airline_ds = Dataset.File.from_files('https://airlinedataset.blob.core.windows.net/airline-10years/*')

In [ ]:
# download the dataset as local files
airline_ds.download(target_path='/local/path')

## Initialize workspace

Load and initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`

In [ ]:
from azureml.core.workspace import Workspace

# if a locally-saved configuration file for the workspace is not available, use the following to load workspace
# ws = Workspace(subscription_id=subscription_id, resource_group=resource_group, workspace_name=workspace_name)

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

datastore = ws.get_default_datastore()
print("Default datastore's name: {}".format(datastore.name))

## Upload data

Upload the dataset to the workspace's default datastore:

In [ ]:
path_on_datastore = 'airline_data'
datastore.upload(src_dir='/local/path', target_path=path_on_datastore, overwrite=False, show_progress=True)

In [ ]:
ds_data = datastore.path(path_on_datastore)
print(ds_data)

## Create AML compute

You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this notebook, we will use Azure ML managed compute ([AmlCompute](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)) for our remote training using a dynamically scalable pool of compute resources.

This notebook will use 10 nodes for hyperparameter optimization, you can modify `max_node` based on available quota in the desired region. Similar to other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. [This article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) includes details on the default limits and how to request more quota.

`vm_size` describes the virtual machine type and size that will be used in the cluster. RAPIDS requires NVIDIA Pascal or newer architecture, you will need to specify compute targets from one of `NC_v2`, `NC_v3`, `ND` or `ND_v2` [GPU virtual machines in Azure](https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-gpu); these are VMs that are provisioned with P40 and V100 GPUs. Let's create an `AmlCompute` cluster of `Standard_NC6s_v3` GPU VMs:

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
gpu_cluster_name = 'gpu-cluster'

if gpu_cluster_name in ws.compute_targets:
    gpu_cluster = ws.compute_targets[gpu_cluster_name]
    if gpu_cluster and type(gpu_cluster) is AmlCompute:
        print('Found compute target. Will use {0} '.format(gpu_cluster_name))
else:
    print('creating new cluster')
    # m_size parameter below could be modified to one of the RAPIDS-supported VM types
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_NC12s_v3', max_nodes = 1)
    # Use VM types with more than one GPU for multi-GPU option, e.g. Standard_NC12s_v3
    
    # create the cluster
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout 
    # if no min node count is provided it uses the scale settings for the cluster
    gpu_cluster.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
# use get_status() to get a detailed status for the current cluster 
print(gpu_cluster.get_status().serialize())

## Prepare training script

Create a project directory that will contain code from your local machine that you will need access to on the remote resource. This includes the training script and additional files your training script depends on. In this example, the training script is provided: 
<br>
`train_rapids_RF.py` - entry script for RAPIDS Estimator that includes loading dataset into cuDF data frame, training with Random Forest and inference using cuML.

In [ ]:
import os

project_folder = './train_rapids'
os.makedirs(project_folder, exist_ok=True)

We will log some metrics by using the `Run` object within the training script:

```python
from azureml.core.run import Run
run = Run.get_context()
```
 
We will also log the parameters and highest accuracy the model achieves:

```python
run.log('Accuracy', np.float(accuracy))
```

These run metrics will become particularly important when we begin hyperparameter tuning our model in the 'Tune model hyperparameters' section.

Copy the training script `train_rapids_RF.py` into your project directory:

In [ ]:
import shutil

shutil.copy('../code/train_rapids_XGB.py', project_folder)
shutil.copy('../code/rapids_csp_azure.py', project_folder)

## Train model on the remote compute

Now that you have your data and training script prepared, you are ready to train on your remote compute.

### Create experiment

Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace.

In [ ]:
from azureml.core import Experiment

experiment_name = 'train_rapids'
experiment = Experiment(ws, name=experiment_name)

### Create environment

The [Environment class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment.environment?view=azure-ml-py) allows you to build a Docker image and customize the system that you will use for training. We will build a container image using a RAPIDS container as base image and install necessary packages. This build is necessary only the first time and will take about 15 minutes. The image will be added to your Azure Container Registry and the environment will be cached after the first run, as long as the environment definition remains the same.

In [ ]:
from azureml.core import Environment

# create the environment
rapids_env = Environment('rapids_env')

# create the environment inside a Docker container
rapids_env.docker.enabled = True

# specify docker steps as a string. Alternatively, load the string from a file
dockerfile = """
FROM rapidsai/rapidsai:0.14-cuda10.2-runtime-ubuntu18.04-py3.7
RUN source activate rapids && \
pip install azureml-sdk && \
pip install azureml-widgets
"""
#FROM nvcr.io/nvidia/rapidsai/rapidsai:0.12-cuda10.0-runtime-ubuntu18.04
# FROM rapidsai/rapidsai:0.14-cuda10.2-runtime-ubuntu18.04-py3.7

# set base image to None since the image is defined by dockerfile
rapids_env.docker.base_image = None
rapids_env.docker.base_dockerfile = dockerfile

# use rapids environment in the container
rapids_env.python.user_managed_dependencies = True

In [ ]:
# from azureml.core.container_registry import ContainerRegistry

# # this is an image available on Docker Hub
# image_name = 'zronaghi/rapidsai-nightly:0.13-cuda10.0-runtime-ubuntu18.04-py3.7-azuresdk-030920'

# # use rapids environment, don't build a new conda environment
# user_managed_dependencies = True

### Create a RAPIDS Estimator

The [Estimator](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.estimator.estimator?view=azure-ml-py) class can be used with machine learning frameworks that do not have a pre-configure estimator. 

`script_params` is a dictionary of command-line arguments to pass to the training script.

In [ ]:
from azureml.train.estimator import Estimator

script_params = {
    '--data_dir': ds_data.as_mount(),
    '--num_boost_round': 10,
    '--max_depth': 6,
    '--learning_rate': 0.3,
}

estimator = Estimator(source_directory=project_folder,
                      script_params=script_params,
                      compute_target=gpu_cluster, 
                      entry_script='train_rapids_XGB.py',
                      environment_definition=rapids_env)
#                       custom_docker_image=image_name,
#                       user_managed=user_managed_dependencies

Run your experiment by submitting your estimator object.

In [ ]:
# start the HyperDrive run
run = experiment.submit(estimator)

## Monitor your run

Monitor and view the progress of the run with a [Notebook widget](https://docs.microsoft.com/en-us/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py).The widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
# run.cancel()

## Delete cluster

In [ ]:
# delete the cluster
# gpu_cluster.delete()